In [1]:
#################################################
## Process the MIMIC-III data files and create a single tabular mixed-data-type object (stored as a Pandas DataFrame)
##
## Input data tables
##  1) PATIENTS
##  2) ADMISSIONS
##  3) DIAGNOSES_ICD
##  4) NOTEEVENTS
##
## Process and create a single table containing
##  1) Clinical note features (discharge note; conditional on surviving most recent admit)
##  2) Demographic features (age at admit, sex at admit)
##  3) Clinical features (number previous admissions --- over study interval)
##  4) Outcomes
##     a) Most responsible ICD code for visit (multinomial/categorical)
##     b) All ICD codes collected over visit (multi-label)
##     c) Death - after discharge (binary)
##     d) Death - 30d after discharge (binary)
##     e) Time to Death/Censor (survival)
##
## Author: Christopher Meaney
## Date: January 2024
#################################################

In [2]:
###################
## Package dependencies
###################

## Numerics
import numpy as np

## Data wrangling
import pandas as pd

## Stats
!pip install scipy
from scipy.stats import ttest_ind

## Environment characteristics
!pip install session_info
import session_info

In [3]:
###################
## Jupyter display options
###################
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
###################
## Import data files
###################

In [5]:
## Filepaths
datadir = "C:/Users/ChristopherMeaney/Desktop/tmp/Vector_SSL/Data/"

patient_path = datadir + "PATIENTS.CSV"
admission_path = datadir + "ADMISSIONS.CSV"
note_path = datadir + "NOTEEVENTS.CSV"
icd_path = datadir + "DIAGNOSES_ICD.CSV"
icd_text_path = datadir + "D_ICD_DIAGNOSES.CSV"

In [6]:
###################
## Output data paths
###################

In [7]:
## Filepaths
datadir = "C:/Users/ChristopherMeaney/Desktop/tmp/Vector_SSL/Data/"

X_csv_path = datadir + "X.csv"
X_pkl_path = datadir + "X.pkl"

In [8]:
############
## Patient table
############
patient = pd.read_csv(filepath_or_buffer=patient_path, sep=",", header=0)
patient.shape

(46520, 8)

In [9]:
patient.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [10]:
## Number unique patient
patient.SUBJECT_ID.nunique()

46520

In [11]:
##
## Distirbution Age
##
## Note "Age" calculation requires specification of an INDEX/REFERENCE-point
## For example: "Age at Admission", "Age at Discharge", "Age at Death", etc. etc.
## The index dates (admission, discharge, etc.) are in the ADMISSSION table
##

In [12]:
## Distribution Sex
patient.GENDER.value_counts(dropna=False)

M    26121
F    20399
Name: GENDER, dtype: int64

In [13]:
## Distribution Death
patient.EXPIRE_FLAG.value_counts(dropna=False)

0    30761
1    15759
Name: EXPIRE_FLAG, dtype: int64

In [14]:
## Another way to look at Death
patient.DOD.isna().value_counts(dropna=False)

True     30761
False    15759
Name: DOD, dtype: int64

In [15]:
## Death in Hospital
patient.DOD_HOSP.isna().value_counts(dropna=False)

True     36546
False     9974
Name: DOD_HOSP, dtype: int64

In [16]:
## Death in SSN (social security database)
patient.DOD_SSN.isna().value_counts(dropna=False)

True     33142
False    13378
Name: DOD_SSN, dtype: int64

In [17]:
## Crosstab of death variables
pd.crosstab(patient.DOD.isna(), patient.EXPIRE_FLAG)

EXPIRE_FLAG,0,1
DOD,,
False,0,15759
True,30761,0


In [18]:
## Threeway crosstab
pd.crosstab(patient.DOD_HOSP.isna(), patient.DOD_SSN.isna())

DOD_SSN,False,True
DOD_HOSP,,
False,7593,2381
True,5785,30761


In [19]:
## Check math on above crosstab
pd.Series(patient.DOD_HOSP.isna() & patient.DOD_SSN.isna()).value_counts()

True     30761
False    15759
dtype: int64

In [20]:
## Create some flags
patient.DOD_HOSP.isna().value_counts()

True     36546
False     9974
Name: DOD_HOSP, dtype: int64

In [21]:
##
## Drop people with indication of death in HOSPITAL (you need to survive your [LAST/MOST-RECENT] hospital stay)
##
patient_sm = patient[patient.DOD_HOSP.isna()==True]
patient_sm.shape

(36546, 8)

In [22]:
## Only keep subset of variable
patient_sm_ = patient_sm.loc[:,["SUBJECT_ID","GENDER","DOB","DOD"]]
patient_sm_.shape

(36546, 4)

In [23]:
## Create death variable/flag
patient_sm_["DOD_FLAG"] = patient_sm_.DOD.isna()
patient_sm_.DOD_FLAG.value_counts()

True     30761
False     5785
Name: DOD_FLAG, dtype: int64

In [24]:
###########
## Admission table
###########
admission = pd.read_csv(filepath_or_buffer=admission_path, sep=",", header=0)
admission.shape

(58976, 19)

In [25]:
admission.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [26]:
## Number unique patients
admission.SUBJECT_ID.nunique()

46520

In [27]:
## Number of unique admissions
admission.HADM_ID.nunique()

58976

In [28]:
## Number of admissions per patient
# admission.SUBJECT_ID.value_counts().value_counts().sort_index(False)

In [29]:
## Number admissions per subject
num_admit_df = pd.DataFrame(admission.SUBJECT_ID.value_counts())

num_admit_df_ = pd.DataFrame({"SUBJECT_ID": num_admit_df.index.to_list(),
                            "NUM_ADMIT": num_admit_df.iloc[:,0]})

num_admit_df_.head()

,SUBJECT_ID,NUM_ADMIT
13033,13033,42
11861,11861,34
109,109,34
5060,5060,31
20643,20643,24


In [30]:
num_admit_df_.NUM_ADMIT.value_counts().sort_index(False)

1     38983
2      5160
3      1342
4       508
5       246
6       113
7        51
8        31
9        26
10       14
11       13
12        8
13        5
14        4
15        1
16        2
17        3
19        1
20        1
21        1
22        1
23        1
24        1
31        1
34        2
42        1
Name: NUM_ADMIT, dtype: int64

In [31]:
## Admission type
admission.ADMISSION_TYPE.value_counts()

EMERGENCY    42071
NEWBORN       7863
ELECTIVE      7706
URGENT        1336
Name: ADMISSION_TYPE, dtype: int64

In [32]:
## Diagnosis
##
## Note: there are many unique diagnoses, and perhaps duplicated concepts --- focus on ICD data in table below...

pd.DataFrame(admission.DIAGNOSIS.value_counts(dropna=False)).head(500)

,DIAGNOSIS
NEWBORN,7823
PNEUMONIA,1566
SEPSIS,1184
CONGESTIVE HEART FAILURE,928
CORONARY ARTERY DISEASE,840
CHEST PAIN,778
INTRACRANIAL HEMORRHAGE,713
ALTERED MENTAL STATUS,712
GASTROINTESTINAL BLEED,686
CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS GRAFT /SDA,583


In [33]:
## Insurance
admission.INSURANCE.value_counts(dropna=False)

Medicare      28215
Private       22582
Medicaid       5785
Government     1783
Self Pay        611
Name: INSURANCE, dtype: int64

In [34]:
## Language
admission.LANGUAGE.value_counts(dropna=False)

ENGL    29086
NaN     25332
SPAN     1083
RUSS      790
PTUN      628
CANT      413
PORT      342
CAPE      256
MAND      156
HAIT      150
ITAL      124
VIET       94
GREE       75
ARAB       47
PERS       44
CAMB       37
POLI       34
AMER       31
HIND       24
KORE       23
ALBA       17
FREN       16
SOMA       13
THAI       12
ETHI       11
*ARM        9
*GUJ        7
LAOT        7
*HUN        7
*YID        7
*CHI        7
*BEN        7
*BUL        5
URDU        5
*CDI        4
*IBO        4
*BUR        4
*URD        4
JAPA        3
**TO        3
TAGA        3
*TEL        3
*MAN        3
*TOI        2
*LEB        2
* BE        2
**SH        2
*FUL        2
*MOR        2
*CAN        2
TURK        2
*TOY        2
BENG        2
*DUT        2
*KHM        2
*AMH        2
GERM        1
*RUS        1
*SPA        1
*PHI        1
** T        1
*CRE        1
*DEA        1
*YOR        1
*PER        1
*LIT        1
SERB        1
*BOS        1
*FIL        1
*ARA        1
*ROM        1
* FU  

In [35]:
## Religion
admission.RELIGION.value_counts(dropna=False)

CATHOLIC                  20606
NOT SPECIFIED             11753
UNOBTAINABLE               8269
PROTESTANT QUAKER          7134
JEWISH                     5314
OTHER                      2696
EPISCOPALIAN                774
GREEK ORTHODOX              459
NaN                         458
CHRISTIAN SCIENTIST         429
BUDDHIST                    267
MUSLIM                      225
JEHOVAH'S WITNESS           139
UNITARIAN-UNIVERSALIST      124
HINDU                       113
ROMANIAN EAST. ORTH          83
7TH DAY ADVENTIST            81
BAPTIST                      28
HEBREW                       16
METHODIST                     7
LUTHERAN                      1
Name: RELIGION, dtype: int64

In [36]:
## Marital Status
admission.MARITAL_STATUS.value_counts(dropna=False)

MARRIED              24239
SINGLE               13254
NaN                  10128
WIDOWED               7211
DIVORCED              3213
SEPARATED              571
UNKNOWN (DEFAULT)      345
LIFE PARTNER            15
Name: MARITAL_STATUS, dtype: int64

In [37]:
## Ethnicity
admission.ETHNICITY.value_counts(dropna=False)

WHITE                                                       40996
BLACK/AFRICAN AMERICAN                                       5440
UNKNOWN/NOT SPECIFIED                                        4523
HISPANIC OR LATINO                                           1696
OTHER                                                        1512
ASIAN                                                        1509
UNABLE TO OBTAIN                                              814
PATIENT DECLINED TO ANSWER                                    559
ASIAN - CHINESE                                               277
HISPANIC/LATINO - PUERTO RICAN                                232
BLACK/CAPE VERDEAN                                            200
WHITE - RUSSIAN                                               164
MULTI RACE ETHNICITY                                          130
BLACK/HAITIAN                                                 101
ASIAN - ASIAN INDIAN                                           85
WHITE - OT

In [38]:
## Hospital expire flag
admission.HOSPITAL_EXPIRE_FLAG.value_counts(dropna=False)

0    53122
1     5854
Name: HOSPITAL_EXPIRE_FLAG, dtype: int64

In [39]:
## Discharge time --- everyone has a discharge time
admission.DISCHTIME.isna().value_counts(dropna=False)

False    58976
Name: DISCHTIME, dtype: int64

In [40]:
##
## Only consider emergency admissions
##
admission_sm = admission[admission.ADMISSION_TYPE=="EMERGENCY"]
admission_sm.shape

(42071, 19)

In [41]:
##
## Only consider emergency visit where person survives --- .tail(1) will take the most recent discharge 
## 
admission_sm_ = admission_sm.sort_values('DISCHTIME').groupby('SUBJECT_ID').tail(1)
admission_sm_.shape

(32610, 19)

In [42]:
admission_sm_.SUBJECT_ID.nunique()

32610

In [43]:
admission_sm_.HADM_ID.nunique()

32610

In [44]:
##
## Subset of columns
##
admission_sm__ = admission_sm_.loc[:,["SUBJECT_ID","HADM_ID","ADMITTIME","DISCHTIME","DEATHTIME"]]
admission_sm__.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME
54531,82574,118464,2100-06-07 19:59:00,2100-06-09 17:09:00,NaN
25039,21081,159656,2100-06-14 12:02:00,2100-06-17 14:20:00,NaN
36814,29156,161773,2100-06-09 01:39:00,2100-06-19 08:15:00,2100-06-19 08:15:00
11374,12001,173927,2100-06-14 04:55:00,2100-06-27 12:00:00,NaN
40630,32096,158366,2100-06-22 03:04:00,2100-06-30 11:35:00,NaN


In [45]:
###########
## Join the patient/admission tables
###########
patient_admission = pd.merge(patient_sm_, admission_sm__, left_on='SUBJECT_ID', right_on='SUBJECT_ID', how='inner')
patient_admission.shape

(23470, 9)

In [46]:
patient_admission.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOD_FLAG,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME
0,249,F,2075-03-13 00:00:00,NaN,True,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,NaN
1,251,M,2090-03-15 00:00:00,NaN,True,117937,2110-07-27 06:46:00,2110-07-29 15:23:00,NaN
2,252,M,2078-03-06 00:00:00,NaN,True,193470,2133-08-15 04:23:00,2133-08-19 17:30:00,NaN
3,253,F,2089-11-26 00:00:00,NaN,True,176189,2174-01-21 20:58:00,2174-01-26 16:15:00,NaN
4,256,M,2086-07-31 00:00:00,NaN,True,108811,2170-08-16 13:55:00,2170-08-22 18:00:00,NaN


In [47]:
patient_admission.DOD_FLAG.value_counts(dropna=False)

True     18513
False     4957
Name: DOD_FLAG, dtype: int64

In [48]:
#############
## Transform many of the date/time variables
#############

In [49]:
## Create age at admission
patient_admission['ADMIT_YEAR'] = patient_admission.ADMITTIME.str.slice(start=0, stop=4)
patient_admission['ADMIT_MONTH'] = patient_admission.ADMITTIME.str.slice(start=5, stop=7)
patient_admission['ADMIT_DAY'] = patient_admission.ADMITTIME.str.slice(start=8, stop=11)

patient_admission['DOB_YEAR'] = patient_admission.DOB.str.slice(start=0, stop=4)
patient_admission['DOB_MONTH'] = patient_admission.DOB.str.slice(start=5, stop=7)
patient_admission['DOB_DAY'] = patient_admission.DOB.str.slice(start=8, stop=11)

patient_admission['DOD_YEAR'] = patient_admission.DOD.str.slice(start=0, stop=4)
patient_admission['DOD_MONTH'] = patient_admission.DOD.str.slice(start=5, stop=7)
patient_admission['DOD_DAY'] = patient_admission.DOD.str.slice(start=8, stop=11)

patient_admission['DISCH_YEAR'] = patient_admission.DISCHTIME.str.slice(start=0, stop=4)
patient_admission['DISCH_MONTH'] = patient_admission.DISCHTIME.str.slice(start=5, stop=7)
patient_admission['DISCH_DAY'] = patient_admission.DISCHTIME.str.slice(start=8, stop=11)

In [50]:
## Admissions 
#patient_admission.ADMIT_YEAR.value_counts()
#patient_admission.ADMIT_MONTH.value_counts()
#patient_admission.ADMIT_DAY.value_counts()

In [51]:
## DOB 
#patient_admission.DOB_YEAR.value_counts()
#patient_admission.DOB_MONTH.value_counts()
#patient_admission.DOB_DAY.value_counts()

In [52]:
## DOD
#patient_admission.DOD_YEAR.value_counts()
#patient_admission.DOD_MONTH.value_counts()
#patient_admission.DOD_DAY.value_counts()

In [53]:
## Discharge
#patient_admission.DISCH_YEAR.value_counts()
#patient_admission.DISCH_MONTH.value_counts()
#patient_admission.DISCH_DAY.value_counts()

In [54]:
## Admission date
patient_admission["ADMIT_DATE"] = pd.to_datetime(pd.DataFrame({'year': patient_admission.ADMIT_YEAR,
                                                                'month': patient_admission.ADMIT_MONTH,
                                                                 'day': patient_admission.ADMIT_DAY})).dt.date

patient_admission.ADMIT_DATE.describe()

count          23470
unique         17317
top       2173-05-26
freq               6
Name: ADMIT_DATE, dtype: object

In [55]:
## DOB date
patient_admission["DOB_DATE"] = pd.to_datetime(pd.DataFrame({'year': patient_admission.DOB_YEAR,
                                                                'month': patient_admission.DOB_MONTH,
                                                                 'day': patient_admission.DOB_DAY})).dt.date

patient_admission.DOB_DATE.describe()

count          23470
unique         18925
top       2099-06-29
freq               6
Name: DOB_DATE, dtype: object

In [56]:
## Death Date
patient_admission["DOD_DATE"] = pd.to_datetime(pd.DataFrame({'year': patient_admission.DOD_YEAR,
                                                                'month': patient_admission.DOD_MONTH,
                                                                 'day': patient_admission.DOD_DAY})).dt.date

patient_admission.DOD_DATE.describe()

count           4957
unique          4653
top       2154-01-22
freq               3
Name: DOD_DATE, dtype: object

In [57]:
## Discharge Date
patient_admission["DISCH_DATE"] = pd.to_datetime(pd.DataFrame({'year': patient_admission.DISCH_YEAR,
                                                                'month': patient_admission.DISCH_MONTH,
                                                                 'day': patient_admission.DISCH_DAY})).dt.date

patient_admission.DISCH_DATE.describe()

count          23470
unique         17101
top       2163-12-30
freq               6
Name: DISCH_DATE, dtype: object

In [58]:
patient_admission.dtypes

SUBJECT_ID      int64
GENDER         object
DOB            object
DOD            object
DOD_FLAG         bool
HADM_ID         int64
ADMITTIME      object
DISCHTIME      object
DEATHTIME      object
ADMIT_YEAR     object
ADMIT_MONTH    object
ADMIT_DAY      object
DOB_YEAR       object
DOB_MONTH      object
DOB_DAY        object
DOD_YEAR       object
DOD_MONTH      object
DOD_DAY        object
DISCH_YEAR     object
DISCH_MONTH    object
DISCH_DAY      object
ADMIT_DATE     object
DOB_DATE       object
DOD_DATE       object
DISCH_DATE     object
dtype: object

In [59]:
## Age Admit
patient_admission['ADMIT_AGE'] = patient_admission.apply(lambda x: (x['ADMIT_DATE'] - x['DOB_DATE']).days/365, axis=1)
patient_admission.ADMIT_AGE.describe()

count    23470.000000
mean        70.054717
std         52.648305
min          0.000000
25%         48.493836
50%         62.832877
75%         76.542466
max        311.767123
Name: ADMIT_AGE, dtype: float64

In [60]:
## Drop persons with ADMIT_AGE>90
## Drop persons with ADMIT_AGE<18
## Only consider adults
patient_admission_sm = patient_admission[patient_admission.ADMIT_AGE<90]
patient_admission_sm = patient_admission_sm[patient_admission_sm.ADMIT_AGE>18]
patient_admission_sm.shape

(22146, 26)

In [61]:
patient_admission_sm.ADMIT_AGE.describe()

count    22146.000000
mean        60.229471
std         17.807487
min         18.008219
25%         48.378082
50%         62.049315
75%         74.930822
max         89.060274
Name: ADMIT_AGE, dtype: float64

In [62]:
patient_admission_sm.DOD_FLAG.value_counts()

True     17750
False     4396
Name: DOD_FLAG, dtype: int64

In [63]:
patient_admission_sm.DOB_DATE = pd.to_datetime(patient_admission_sm.DOB_DATE)
patient_admission_sm.ADMIT_DATE = pd.to_datetime(patient_admission_sm.ADMIT_DATE)
patient_admission_sm.DISCH_DATE = pd.to_datetime(patient_admission_sm.DISCH_DATE)
patient_admission_sm.DOD_DATE = pd.to_datetime(patient_admission_sm.DOD_DATE)

# patient_admission_sm.dtypes

In [64]:
##
## Break into two datasets --- those who die versus those who do not
##
patient_admission_dead = patient_admission_sm[patient_admission_sm.DOD_FLAG==False]
patient_admission_nodead = patient_admission_sm[patient_admission_sm.DOD_FLAG==True]

[patient_admission_dead.shape, patient_admission_nodead.shape]

[(4396, 26), (17750, 26)]

In [65]:
## Time between discharge and death
patient_admission_dead['DISCH_2_DOD'] = patient_admission_dead.apply(lambda x: (x['DOD_DATE'] - x['DISCH_DATE']).days, axis=1)
patient_admission_dead.DISCH_2_DOD.describe()

C:\Users\ChristopherMeaney\AppData\Local\Temp\ipykernel_11404\752542114.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_admission_dead['DISCH_2_DOD'] = patient_admission_dead.apply(lambda x: (x['DOD_DATE'] - x['DISCH_DATE']).days, axis=1)


count    4396.000000
mean      721.833940
std       838.470267
min       -10.000000
25%        74.000000
50%       378.000000
75%      1119.250000
max      4121.000000
Name: DISCH_2_DOD, dtype: float64

In [66]:
patient_admission_dead['DISCH_2_DOD_'] = np.where(patient_admission_dead['DISCH_2_DOD']<0, 0, patient_admission_dead['DISCH_2_DOD'])
patient_admission_dead['DISCH_2_DOD_'].describe()

C:\Users\ChristopherMeaney\AppData\Local\Temp\ipykernel_11404\463395712.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_admission_dead['DISCH_2_DOD_'] = np.where(patient_admission_dead['DISCH_2_DOD']<0, 0, patient_admission_dead['DISCH_2_DOD'])


count    4396.000000
mean      721.836215
std       838.468295
min         0.000000
25%        74.000000
50%       378.000000
75%      1119.250000
max      4121.000000
Name: DISCH_2_DOD_, dtype: float64

In [67]:
## Flag whether death is within 30d of discharge
patient_admission_dead['DISCH_2_DOD_30D'] = patient_admission_dead.DISCH_2_DOD_ < 30
patient_admission_dead['DISCH_2_DOD_30D'].value_counts()

C:\Users\ChristopherMeaney\AppData\Local\Temp\ipykernel_11404\3803103884.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_admission_dead['DISCH_2_DOD_30D'] = patient_admission_dead.DISCH_2_DOD_ < 30


False    3744
True      652
Name: DISCH_2_DOD_30D, dtype: int64

In [68]:
## Upper bound censoring time at 30d
patient_admission_dead['DISCH_2_DOD_30D_SURVTIME'] = np.where(patient_admission_dead.DISCH_2_DOD_<30, patient_admission_dead.DISCH_2_DOD_, 30)
patient_admission_dead['DISCH_2_DOD_30D_SURVTIME'].describe()

C:\Users\ChristopherMeaney\AppData\Local\Temp\ipykernel_11404\3217262777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_admission_dead['DISCH_2_DOD_30D_SURVTIME'] = np.where(patient_admission_dead.DISCH_2_DOD_<30, patient_admission_dead.DISCH_2_DOD_, 30)


count    4396.000000
mean       27.285032
std         7.293425
min         0.000000
25%        30.000000
50%        30.000000
75%        30.000000
max        30.000000
Name: DISCH_2_DOD_30D_SURVTIME, dtype: float64

In [69]:
## Same idea with non-dead people
patient_admission_nodead['DISCH_2_DOD'] = np.nan
patient_admission_nodead['DISCH_2_DOD_'] = np.nan
patient_admission_nodead['DISCH_2_DOD_30D'] = False
patient_admission_nodead['DISCH_2_DOD_30D_SURVTIME'] = 30

C:\Users\ChristopherMeaney\AppData\Local\Temp\ipykernel_11404\1187510960.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_admission_nodead['DISCH_2_DOD'] = np.nan
C:\Users\ChristopherMeaney\AppData\Local\Temp\ipykernel_11404\1187510960.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_admission_nodead['DISCH_2_DOD_'] = np.nan
C:\Users\ChristopherMeaney\AppData\Local\Temp\ipykernel_11404\1187510960.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [70]:
patient_admission_final = pd.concat([patient_admission_dead, patient_admission_nodead], axis=0)
patient_admission_final.shape

(22146, 30)

In [71]:
patient_admission_final.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOD_FLAG,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMIT_YEAR,ADMIT_MONTH,ADMIT_DAY,DOB_YEAR,DOB_MONTH,DOB_DAY,DOD_YEAR,DOD_MONTH,DOD_DAY,DISCH_YEAR,DISCH_MONTH,DISCH_DAY,ADMIT_DATE,DOB_DATE,DOD_DATE,DISCH_DATE,ADMIT_AGE,DISCH_2_DOD,DISCH_2_DOD_,DISCH_2_DOD_30D,DISCH_2_DOD_30D_SURVTIME
8,666,F,2069-08-26 00:00:00,2139-11-20 00:00:00,False,182220,2129-03-26 19:21:00,2129-03-28 11:36:00,NaN,2129,03,26,2069,08,26,2139,11,20,2129,03,28,2129-03-26,2069-08-26,2139-11-20,2129-03-28,59.619178,3889.0,3889.0,False,30
16,682,F,2036-12-26 00:00:00,2119-08-03 00:00:00,False,188382,2118-08-19 11:19:00,2118-09-03 14:40:00,NaN,2118,08,19,2036,12,26,2119,08,03,2118,09,03,2118-08-19,2036-12-26,2119-08-03,2118-09-03,81.698630,334.0,334.0,False,30
20,695,F,2093-05-14 00:00:00,2178-09-16 00:00:00,False,177128,2178-08-05 07:38:00,2178-08-13 14:00:00,NaN,2178,08,05,2093,05,14,2178,09,16,2178,08,13,2178-08-05,2093-05-14,2178-09-16,2178-08-13,85.282192,34.0,34.0,False,30
26,707,F,2026-04-05 00:00:00,2116-01-04 00:00:00,False,140286,2115-02-28 21:20:00,2115-03-06 13:25:00,NaN,2115,02,28,2026,04,05,2116,01,04,2115,03,06,2115-02-28,2026-04-05,2116-01-04,2115-03-06,88.958904,304.0,304.0,False,30
28,711,M,2100-03-06 00:00:00,2185-05-26 00:00:00,False,158767,2185-03-22 09:47:00,2185-05-16 17:10:00,NaN,2185,03,22,2100,03,06,2185,05,26,2185,05,16,2185-03-22,2100-03-06,2185-05-26,2185-05-16,85.101370,10.0,10.0,True,10


In [72]:
keep_vars = ["SUBJECT_ID",
            "HADM_ID",
            "DOB", "DOB_DATE",
            "ADMITTIME", "ADMIT_DATE",
            "DISCHTIME", "DISCH_DATE",
            "DOD", "DOD_FLAG", "DOD_DATE",
            "GENDER",
            "ADMIT_AGE",
            "DISCH_2_DOD_",
            "DISCH_2_DOD_30D",
            "DISCH_2_DOD_30D_SURVTIME"
            ]

patient_admission_final_sm = patient_admission_final[keep_vars]
#patient_admission_final_sm.head()

In [73]:
patient_admission_final_sm_ = pd.merge(patient_admission_final_sm, num_admit_df_, left_on='SUBJECT_ID', right_on='SUBJECT_ID', how='left')
patient_admission_final_sm_.head()

,SUBJECT_ID,HADM_ID,DOB,DOB_DATE,ADMITTIME,ADMIT_DATE,DISCHTIME,DISCH_DATE,DOD,DOD_FLAG,DOD_DATE,GENDER,ADMIT_AGE,DISCH_2_DOD_,DISCH_2_DOD_30D,DISCH_2_DOD_30D_SURVTIME,NUM_ADMIT
0,666,182220,2069-08-26 00:00:00,2069-08-26,2129-03-26 19:21:00,2129-03-26,2129-03-28 11:36:00,2129-03-28,2139-11-20 00:00:00,False,2139-11-20,F,59.619178,3889.0,False,30,1
1,682,188382,2036-12-26 00:00:00,2036-12-26,2118-08-19 11:19:00,2118-08-19,2118-09-03 14:40:00,2118-09-03,2119-08-03 00:00:00,False,2119-08-03,F,81.698630,334.0,False,30,1
2,695,177128,2093-05-14 00:00:00,2093-05-14,2178-08-05 07:38:00,2178-08-05,2178-08-13 14:00:00,2178-08-13,2178-09-16 00:00:00,False,2178-09-16,F,85.282192,34.0,False,30,2
3,707,140286,2026-04-05 00:00:00,2026-04-05,2115-02-28 21:20:00,2115-02-28,2115-03-06 13:25:00,2115-03-06,2116-01-04 00:00:00,False,2116-01-04,F,88.958904,304.0,False,30,1
4,711,158767,2100-03-06 00:00:00,2100-03-06,2185-03-22 09:47:00,2185-03-22,2185-05-16 17:10:00,2185-05-16,2185-05-26 00:00:00,False,2185-05-26,M,85.101370,10.0,True,10,5


In [74]:
# patient_admission_final_sm_.NUM_ADMIT.value_counts(dropna=False)

In [75]:
###########
## ICD table
###########
icd = pd.read_csv(filepath_or_buffer=icd_path, sep=",", header=0)
icd.shape

(651047, 5)

In [76]:
icd.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [77]:
## Unique Admissions
icd.HADM_ID.nunique()

58976

In [78]:
## Get most responsible code --- sort by and head(1) grabs the most responsible code
icd_mrp = icd.sort_values(['SEQ_NUM']).groupby('HADM_ID').head(1)
icd_mrp_sm = icd_mrp[["HADM_ID","ICD9_CODE"]]
icd_mrp_sm.head()

,HADM_ID,ICD9_CODE
0,172335,40301
111750,176956,86501
111742,148755,99674
519073,102891,0389
335323,166021,44100


In [79]:
## Merge into patient_admission file
patient_admission_final_sm__ = pd.merge(patient_admission_final_sm_, icd_mrp_sm, left_on='HADM_ID', right_on='HADM_ID', how='left')
# patient_admission_final_sm__.head()

In [80]:
## Number of codes per person
icd_list = icd.groupby('HADM_ID')['ICD9_CODE'].apply(list)
# icd_list.head()

## DataFrame of ICD Codes (list form); and admission IDs
icd_list_df = pd.DataFrame({
    "HADM_ID": icd_list.index.to_list(),
    "ICD_LIST": icd_list
})

icd_list_df = icd_list_df.reset_index(drop=True)
icd_list_df.columns = ["HADM_ID", "ICD9_CODE_LIST"]
# icd_list_df.head()

In [81]:
## Merge into patient_admission file
patient_admission_final_sm___ = pd.merge(patient_admission_final_sm__, icd_list_df, left_on='HADM_ID', right_on='HADM_ID', how='left')
# patient_admission_final_sm___.head()

In [82]:
## How many codes per person/admission
patient_admission_final_sm___['NUM_ICD9_CODE'] = [len(x) for x in patient_admission_final_sm___['ICD9_CODE_LIST']]
# pd.Series(len_list).value_counts().sort_index()

patient_admission_final_sm___.head()

,SUBJECT_ID,HADM_ID,DOB,DOB_DATE,ADMITTIME,ADMIT_DATE,DISCHTIME,DISCH_DATE,DOD,DOD_FLAG,DOD_DATE,GENDER,ADMIT_AGE,DISCH_2_DOD_,DISCH_2_DOD_30D,DISCH_2_DOD_30D_SURVTIME,NUM_ADMIT,ICD9_CODE,ICD9_CODE_LIST,NUM_ICD9_CODE
0,666,182220,2069-08-26 00:00:00,2069-08-26,2129-03-26 19:21:00,2129-03-26,2129-03-28 11:36:00,2129-03-28,2139-11-20 00:00:00,False,2139-11-20,F,59.619178,3889.0,False,30,1,431,"[431, V4581, 25000, 4019, 2724]",5
1,682,188382,2036-12-26 00:00:00,2036-12-26,2118-08-19 11:19:00,2118-08-19,2118-09-03 14:40:00,2118-09-03,2119-08-03 00:00:00,False,2119-08-03,F,81.698630,334.0,False,30,1,4472,"[4472, 41071, 42830, 5990, 4280, 5849, 40391, ...",9
2,695,177128,2093-05-14 00:00:00,2093-05-14,2178-08-05 07:38:00,2178-08-05,2178-08-13 14:00:00,2178-08-13,2178-09-16 00:00:00,False,2178-09-16,F,85.282192,34.0,False,30,2,5771,"[5771, 48241, 42731, 2765, 2762, 4240, 3970, 4...",9
3,707,140286,2026-04-05 00:00:00,2026-04-05,2115-02-28 21:20:00,2115-02-28,2115-03-06 13:25:00,2115-03-06,2116-01-04 00:00:00,False,2116-01-04,F,88.958904,304.0,False,30,1,486,"[486, 51881, 4280, 49121, 78559, 42731, 2765, ...",9
4,711,158767,2100-03-06 00:00:00,2100-03-06,2185-03-22 09:47:00,2185-03-22,2185-05-16 17:10:00,2185-05-16,2185-05-26 00:00:00,False,2185-05-26,M,85.101370,10.0,True,10,5,0389,"[0389, 78552, 51881, 4870, 48241, 4820, 1122, ...",19


In [83]:
###########
## Note table
###########

In [84]:
note = pd.read_csv(filepath_or_buffer=note_path, sep=",", header=0)
note.shape

C:\Users\ChristopherMeaney\anaconda3\envs\vector_ssl\lib\site-packages\IPython\core\interactiveshell.py:3269: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(2083180, 11)

In [85]:
note.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [86]:
## Note type
note.CATEGORY.value_counts()

Nursing/other        822497
Radiology            522279
Nursing              223556
ECG                  209051
Physician            141624
Discharge summary     59652
Echo                  45794
Respiratory           31739
Nutrition              9418
General                8301
Rehab Services         5431
Social Work            2670
Case Management         967
Pharmacy                103
Consult                  98
Name: CATEGORY, dtype: int64

In [87]:
#########
## Only keep the discharge summaries
#########
note_dc = note[note.CATEGORY=="Discharge summary"]
note_dc.shape

(59652, 11)

In [88]:
## Keep subset of variables
note_dc_sm = note_dc.loc[:,["HADM_ID","CHARTDATE","CATEGORY","TEXT"]]
note_dc_sm.shape

(59652, 4)

In [89]:
## How long are the notes (discharge summaries in particular)
note_dc_sm['TEXT_LEN'] = [len(x) for x in note_dc_sm.TEXT]
note_dc_sm.TEXT_LEN.describe()

count    59652.000000
mean      9618.916801
std       5539.637550
min         54.000000
25%       5841.000000
50%       8878.000000
75%      12590.000000
max      55728.000000
Name: TEXT_LEN, dtype: float64

In [90]:
## How many discharge notes per HADM_ID
note_dc_sm.HADM_ID.value_counts().value_counts()
##
## As expected...some HADM_ID have multiple discharge notes
##

1    47006
2     4782
3      732
4      161
5       30
6       13
7        2
Name: HADM_ID, dtype: int64

In [91]:
## Combine text strings (multiple discharge notes) by unique HADM_ID (cuz some admission have multiple discharge notes)
note_dc_sm_ = note_dc_sm.groupby('HADM_ID').TEXT.apply(' '.join).reset_index()
note_dc_sm_.head()

,HADM_ID,TEXT
0,100001.0,Admission Date: [**2117-9-11**] ...
1,100003.0,Admission Date: [**2150-4-17**] ...
2,100006.0,Admission Date: [**2108-4-6**] Discharg...
3,100007.0,Admission Date: [**2145-3-31**] ...
4,100009.0,Admission Date: [**2162-5-16**] ...


In [92]:
## Check that HADM_ID is unique now
note_dc_sm_.HADM_ID.value_counts().value_counts()

1    52726
Name: HADM_ID, dtype: int64

In [93]:
## How long are the notes (discharge summaries in particular)
note_dc_sm_['TEXT_LEN'] = [len(x) for x in note_dc_sm_.TEXT]
note_dc_sm_.TEXT_LEN.describe()

count    52726.000000
mean     10882.573133
std       5560.607915
min        528.000000
25%       6926.000000
50%       9875.000000
75%      13628.750000
max      78231.000000
Name: TEXT_LEN, dtype: float64

In [94]:
#########
## Merge into patient_admission_icd table
#########

In [95]:
X = pd.merge(patient_admission_final_sm___, note_dc_sm_, left_on='HADM_ID', right_on='HADM_ID', how='inner')
[patient_admission_final_sm___.shape, note_dc_sm.shape, X.shape]

[(22146, 20), (59652, 5), (21460, 22)]

In [96]:
## Continue transforming the final data structure
final_keep_vars = [
            ## ID variables
            "SUBJECT_ID",
            "HADM_ID",
            ## Date variables (denoting major events)
            "DOB_DATE",
            "ADMIT_DATE",
            "DISCH_DATE",
            "DOD_DATE", "DOD_FLAG", 
            ## Predictors of outcome variables (aka. features)
            "GENDER",
            "ADMIT_AGE",
            "NUM_ADMIT",
            ## ICD9 code related outcomes (coded following discharge)
            "ICD9_CODE", "ICD9_CODE_LIST", "NUM_ICD9_CODE",
            ## Time to event/death related outcomes (for people who died following discharge)
            "DISCH_2_DOD_",
            "DISCH_2_DOD_30D",
            "DISCH_2_DOD_30D_SURVTIME",
            ## Text predictors/features (all discharge summaries --- note again, all lived HOSP and were hence discharged, most with note)
            # "CHARTDATE", "CATEGORY", 
            "TEXT", "TEXT_LEN"
            ]


X = X[final_keep_vars]

In [97]:
X.head()

,SUBJECT_ID,HADM_ID,DOB_DATE,ADMIT_DATE,DISCH_DATE,DOD_DATE,DOD_FLAG,GENDER,ADMIT_AGE,NUM_ADMIT,ICD9_CODE,ICD9_CODE_LIST,NUM_ICD9_CODE,DISCH_2_DOD_,DISCH_2_DOD_30D,DISCH_2_DOD_30D_SURVTIME,TEXT,TEXT_LEN
0,682,188382,2036-12-26,2118-08-19,2118-09-03,2119-08-03,False,F,81.698630,1,4472,"[4472, 41071, 42830, 5990, 4280, 5849, 40391, ...",9,334.0,False,30,Admission Date: [**2118-8-19**] ...,10041
1,695,177128,2093-05-14,2178-08-05,2178-08-13,2178-09-16,False,F,85.282192,2,5771,"[5771, 48241, 42731, 2765, 2762, 4240, 3970, 4...",9,34.0,False,30,Admission Date: [**2178-8-5**] Discharg...,9829
2,707,140286,2026-04-05,2115-02-28,2115-03-06,2116-01-04,False,F,88.958904,1,486,"[486, 51881, 4280, 49121, 78559, 42731, 2765, ...",9,304.0,False,30,Admission Date: [**2115-2-28**] Dischar...,8178
3,711,158767,2100-03-06,2185-03-22,2185-05-16,2185-05-26,False,M,85.101370,5,0389,"[0389, 78552, 51881, 4870, 48241, 4820, 1122, ...",19,10.0,True,10,Admission Date: [**2185-3-22**] ...,19947
4,717,135596,2038-05-22,2116-02-04,2116-02-17,2119-03-22,False,F,77.756164,1,41401,"[41401, 4111, 42731, 4019, 7140, 2720]",6,1129.0,False,30,Admission Date: [**2116-2-4**] D...,9423


In [98]:
## Flag --- maybe a coding error; or maybe the charting occurred after the actual discharge??
# pd.Series(X.CHARTDATE <= X.DISCH_DATE).value_counts()

In [99]:
##
## FINAL TOUCH --- Add ICD 9 Text Labels
##
icd_text = pd.read_csv(icd_text_path, sep=",", header=0)
icd_text.head()

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."


In [100]:
icd_text_sm = icd_text[["ICD9_CODE","SHORT_TITLE","LONG_TITLE"]]

In [101]:
X = pd.merge(X, icd_text_sm, left_on='ICD9_CODE', right_on='ICD9_CODE', how='inner')
X.head()

,SUBJECT_ID,HADM_ID,DOB_DATE,ADMIT_DATE,DISCH_DATE,DOD_DATE,DOD_FLAG,GENDER,ADMIT_AGE,NUM_ADMIT,ICD9_CODE,ICD9_CODE_LIST,NUM_ICD9_CODE,DISCH_2_DOD_,DISCH_2_DOD_30D,DISCH_2_DOD_30D_SURVTIME,TEXT,TEXT_LEN,SHORT_TITLE,LONG_TITLE
0,682,188382,2036-12-26,2118-08-19,2118-09-03,2119-08-03,False,F,81.698630,1,4472,"[4472, 41071, 42830, 5990, 4280, 5849, 40391, ...",9,334.0,False,30,Admission Date: [**2118-8-19**] ...,10041,Rupture of artery,Rupture of artery
1,786,117381,2033-05-14,2115-12-29,2116-01-10,2116-08-31,False,M,82.679452,1,4472,"[4472, 4538, 00845, 49121, 42731, 2875, 2851, ...",9,234.0,False,30,Admission Date: [**2115-12-29**] Discharg...,14642,Rupture of artery,Rupture of artery
2,16181,190902,2116-07-18,2196-02-14,2196-02-17,2202-11-22,False,M,79.630137,1,4472,"[4472, 496, 4019, 4422, 53081, 3051, V173, V12...",9,2469.0,False,30,Admission Date: [**2196-2-14**] ...,10935,Rupture of artery,Rupture of artery
3,60809,131743,2048-08-27,2123-04-08,2123-05-01,NaT,True,F,74.660274,1,4472,"[4472, 5849, 59382, 591, 5934, 2851, E8792, 59...",16,NaN,False,30,Admission Date: [**2123-4-8**] D...,9302,Rupture of artery,Rupture of artery
4,695,177128,2093-05-14,2178-08-05,2178-08-13,2178-09-16,False,F,85.282192,2,5771,"[5771, 48241, 42731, 2765, 2762, 4240, 3970, 4...",9,34.0,False,30,Admission Date: [**2178-8-5**] Discharg...,9829,Chronic pancreatitis,Chronic pancreatitis


In [102]:
##################################################
##
## Inspect some of the univariate properties of the final dataset
##
##################################################

In [103]:
## Dimensions
X.shape

(21177, 20)

In [104]:
## Unique subject check
X.SUBJECT_ID.nunique()

21177

In [105]:
## Unique admission check
X.HADM_ID.nunique()

21177

In [106]:
## Gender
X.GENDER.value_counts(dropna=False)

M    12235
F     8942
Name: GENDER, dtype: int64

In [107]:
## Age
X.ADMIT_AGE.describe()

count    21177.000000
mean        60.645928
std         17.643485
min         18.008219
25%         49.065753
50%         62.446575
75%         75.189041
max         89.060274
Name: ADMIT_AGE, dtype: float64

In [108]:
## Number previous admissions
(X.NUM_ADMIT-1).value_counts(dropna=False).sort_index(False)

0     17231
1      2749
2       697
3       254
4       117
5        45
6        27
7        16
8        11
9         5
10        6
11        3
12        3
13        3
14        1
15        1
16        3
19        1
20        1
21        1
22        1
33        1
Name: NUM_ADMIT, dtype: int64

In [109]:
## Text Length
X.TEXT_LEN.describe()

count    21177.000000
mean     11509.480144
std       5534.749019
min        528.000000
25%       7648.000000
50%      10555.000000
75%      14214.000000
max      54350.000000
Name: TEXT_LEN, dtype: float64

In [110]:
## ICD9 Codes (Most responsible diagnosis)
X.LONG_TITLE.value_counts().head(25)

Coronary atherosclerosis of native coronary artery                                                                1559
Subendocardial infarction, initial episode of care                                                                 920
Unspecified septicemia                                                                                             700
Intracerebral hemorrhage                                                                                           514
Acute respiratory failure                                                                                          403
Subarachnoid hemorrhage                                                                                            341
Acute myocardial infarction of other inferior wall, initial episode of care                                        288
Acute myocardial infarction of other anterior wall, initial episode of care                                        257
Aortic valve disorders                          

In [111]:
## How many unique ICD9 Codes
X.ICD9_CODE.nunique()

1908

In [112]:
## Deaths
X.DOD_FLAG.value_counts(dropna=False)

True     16950
False     4227
Name: DOD_FLAG, dtype: int64

In [113]:
## Discharge to Death (subset of those who died)
X.DISCH_2_DOD_.describe()

count    4227.000000
mean      715.118760
std       833.939394
min         0.000000
25%        72.000000
50%       372.000000
75%      1110.500000
max      4121.000000
Name: DISCH_2_DOD_, dtype: float64

In [114]:
## Flag of whether death (post discharge) occurred within 30d
X.DISCH_2_DOD_30D.value_counts(dropna=False)

False    20542
True       635
Name: DISCH_2_DOD_30D, dtype: int64

In [115]:
## Top ICD9 Codes - in those who die
X.LONG_TITLE[X.DOD_FLAG==False].value_counts().head(10)

Unspecified septicemia                                             219
Coronary atherosclerosis of native coronary artery                 185
Subendocardial infarction, initial episode of care                 167
Acute respiratory failure                                          152
Intracerebral hemorrhage                                           146
Pneumonitis due to inhalation of food or vomitus                   119
Congestive heart failure, unspecified                               91
Pneumonia, organism unspecified                                     67
Cerebral artery occlusion, unspecified with cerebral infarction     54
Acute kidney failure, unspecified                                   53
Name: LONG_TITLE, dtype: int64

In [116]:
## Top ICD9 Codes - in those who DO NOT die
X.LONG_TITLE[X.DOD_FLAG==True].value_counts().head(10)

Coronary atherosclerosis of native coronary artery                             1374
Subendocardial infarction, initial episode of care                              753
Unspecified septicemia                                                          481
Intracerebral hemorrhage                                                        368
Subarachnoid hemorrhage                                                         320
Acute myocardial infarction of other inferior wall, initial episode of care     260
Acute respiratory failure                                                       251
Acute myocardial infarction of other anterior wall, initial episode of care     226
Aortic valve disorders                                                          214
Acute pancreatitis                                                              187
Name: LONG_TITLE, dtype: int64

In [117]:
###################################
##
## Factors associated with death --- the findings appear to have some face validity
##
###################################

In [118]:
## 20% of persons in the cohort die at some time following discharge
X['DOD_FLAG_'] = ~X.DOD_FLAG
X.DOD_FLAG_.value_counts(normalize=True, dropna=False)

False    0.800397
True     0.199603
Name: DOD_FLAG_, dtype: float64

In [119]:
## 3% of persons in the cohort die <30d following discharge
X.DISCH_2_DOD_30D.value_counts(normalize=True, dropna=False)

False    0.970015
True     0.029985
Name: DISCH_2_DOD_30D, dtype: float64

In [120]:
##
## Death versus age
##

In [121]:
X.groupby('DOD_FLAG_').ADMIT_AGE.describe().unstack(0)

       DOD_FLAG_
count  False        16950.000000
       True          4227.000000
mean   False           58.051689
       True            71.048660
std    False           17.512235
       True            13.951204
min    False           18.008219
       True            18.383562
25%    False           46.450000
       True            62.756164
50%    False           59.498630
       True            74.789041
75%    False           71.681507
       True            82.030137
max    False           89.060274
       True            89.054795
dtype: float64

In [122]:
X.groupby('DISCH_2_DOD_30D').ADMIT_AGE.describe().unstack(0)

       DISCH_2_DOD_30D
count  False              20542.000000
       True                 635.000000
mean   False                 60.257211
       True                  73.220753
std    False                 17.636499
       True                  12.509890
min    False                 18.008219
       True                  25.197260
25%    False                 48.613699
       True                  66.871233
50%    False                 62.008219
       True                  76.605479
75%    False                 74.679452
       True                  83.389041
max    False                 89.060274
       True                  89.008219
dtype: float64

In [123]:
##
## Death by gender
##

In [124]:
pd.crosstab(X.GENDER, X.DOD_FLAG_, normalize='columns')

DOD_FLAG_,False,True
GENDER,,
F,0.414572,0.45304
M,0.585428,0.54696


In [125]:
pd.crosstab(X.GENDER, X.DISCH_2_DOD_30D, normalize='columns')

DISCH_2_DOD_30D,False,True
GENDER,,
F,0.421186,0.456693
M,0.578814,0.543307


In [126]:
##
## Death by number previous admissions
##

In [127]:
pd.crosstab(X.NUM_ADMIT-1, X.DOD_FLAG_, normalize='columns').head(5)

DOD_FLAG_,False,True
NUM_ADMIT,,
0,0.820767,0.785190
1,0.126077,0.144784
2,0.030442,0.042820
3,0.011563,0.013721
4,0.005192,0.006861


In [128]:
pd.crosstab(X.NUM_ADMIT-1, X.DISCH_2_DOD_30D, normalize='columns').head(5)

DISCH_2_DOD_30D,False,True
NUM_ADMIT,,
0,0.815695,0.748031
1,0.128615,0.168504
2,0.032178,0.056693
3,0.012122,0.007874
4,0.005404,0.009449


In [129]:
##
## Death versus text length
##

In [130]:
X.groupby('DOD_FLAG_').TEXT_LEN.describe().unstack(0)

       DOD_FLAG_
count  False        16950.000000
       True          4227.000000
mean   False        11455.065841
       True         11727.678022
std    False         5519.962751
       True          5588.986516
min    False          528.000000
       True           587.000000
25%    False         7586.250000
       True          7822.000000
50%    False        10501.000000
       True         10816.000000
75%    False        14129.250000
       True         14487.000000
max    False        53330.000000
       True         54350.000000
dtype: float64

In [131]:
X.groupby('DISCH_2_DOD_30D').TEXT_LEN.describe().unstack(0)

       DISCH_2_DOD_30D
count  False              20542.000000
       True                 635.000000
mean   False              11476.193847
       True               12586.278740
std    False               5520.215231
       True                5889.831360
min    False                528.000000
       True                 587.000000
25%    False               7619.250000
       True                8792.000000
50%    False              10525.000000
       True               11488.000000
75%    False              14159.000000
       True               15181.000000
max    False              54350.000000
       True               46332.000000
dtype: float64

In [132]:
####################################
##
## Serialize final object to disk
##
####################################

In [133]:
## Pickle/serialize the object
X.to_pickle(X_pkl_path)

In [134]:
## Save object to CSV
X.to_csv(X_csv_path)

In [135]:
####################################
##
## Characterize Notebook Environment
## 
####################################

In [136]:
session_info.show()